In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45851
2,Huelva,Confirmados PDIA 14 días,291
3,Huelva,Tasa PDIA 14 días,"56,70635461932693"
4,Huelva,Confirmados PDIA 7 días,127
5,Huelva,Tasa PDIA 7 dias,"24,748134146579105"
6,Huelva,Total Confirmados,46054
7,Huelva,Curados,44926
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45851.0


/tmp/ipykernel_69810/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12614.0


/tmp/ipykernel_69810/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_69810/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_69810/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_69810/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6254 personas en los últimos 7 días 

Un positivo PCR cada 3196 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45851.0,127.0,291.0,513170.0,24.748134,56.706355,29.0
Huelva-Costa,27125.0,77.0,155.0,289548.0,26.593173,53.531711,22.0
Cartaya,2071.0,31.0,62.0,20083.0,154.359408,308.718817,9.0
Huelva (capital),12614.0,23.0,45.0,143837.0,15.990322,31.285413,7.0
Condado-Campiña,14251.0,35.0,105.0,156231.0,22.402724,67.208173,4.0
Moguer,2000.0,15.0,31.0,21867.0,68.596515,141.766132,3.0
Sierra de Huelva-Andévalo Central,4049.0,15.0,30.0,67391.0,22.258165,44.516330,3.0
Isla Cristina,3145.0,6.0,14.0,21393.0,28.046557,65.441967,2.0
Lepe,3034.0,8.0,11.0,27880.0,28.694405,39.454806,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,27.0,5.0,5.0,713.0,701.262272,701.262272,1.0
Villarrasa,232.0,2.0,7.0,2211.0,90.456807,316.598824,0.0
Cartaya,2071.0,31.0,62.0,20083.0,154.359408,308.718817,9.0
Zalamea la Real,120.0,5.0,6.0,3054.0,163.719712,196.463654,1.0
Moguer,2000.0,15.0,31.0,21867.0,68.596515,141.766132,3.0
Palos de la Frontera,1052.0,4.0,16.0,11742.0,34.065747,136.262988,0.0
Aroche,131.0,2.0,3.0,3054.0,65.487885,98.231827,0.0
Palma del Condado (La),1428.0,5.0,10.0,10801.0,46.292010,92.584020,0.0
Valverde del Camino,859.0,1.0,11.0,12750.0,7.843137,86.274510,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,859.0,1.0,11.0,12750.0,7.843137,86.274510,0.0,0.090909
Almonte,2310.0,3.0,14.0,24507.0,12.241400,57.126535,0.0,0.214286
Palos de la Frontera,1052.0,4.0,16.0,11742.0,34.065747,136.262988,0.0,0.250000
Bollullos Par del Condado,1294.0,2.0,8.0,14387.0,13.901439,55.605755,0.0,0.250000
Villarrasa,232.0,2.0,7.0,2211.0,90.456807,316.598824,0.0,0.285714
Aljaraque,1633.0,1.0,3.0,21474.0,4.656794,13.970383,0.0,0.333333
Condado-Campiña,14251.0,35.0,105.0,156231.0,22.402724,67.208173,4.0,0.333333
Punta Umbría,1276.0,3.0,8.0,15355.0,19.537610,52.100293,0.0,0.375000
Isla Cristina,3145.0,6.0,14.0,21393.0,28.046557,65.441967,2.0,0.428571
